Regression Pipeline
===================

This script demonstrates how to preprocess data and run an Optuna study to tune hyperparameters 
for a regression model. The features are sourced from 'media-4.xlsx' (using the sheet 'all features')
while the target values (viability) are loaded from 'secretome_samples_for_RNAseq_170111.xlsx'.

The viability target is a continuous variable, so we modify the pipeline as follows:
  - Load the target from a new Excel file and select the columns ['Gene name', 'Viability (harvest) [%]'].
  - Rename columns so that the gene names match the feature dataframe index and rename the viability column to “Viability”.
  - Remove any sampling or stratification (since the target is continuous).
  - In the Optuna objective, choose among regression models and use an appropriate metric (R²).

In [ ]:
#############################
# Import Libraries & Modules
#############################

import json
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

# Data splitting and imputation
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score

# ML models
import shap
import optuna
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

In [ ]:
###########################
# Define Helper Functions
###########################

def show_heatmap(df, predictor_list, target='Viability', figsize=(20, 10), title='Correlation Heatmap'):
    # Create a correlation matrix (only numeric columns) and plot a heatmap
    correlation_matrix = df[predictor_list + [target]].corr(method='pearson')
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    plt.figure(figsize=figsize)
    plt.title(title)
    sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', center=0, linewidths=0.5)
    plt.show()

def standardize_values(X):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X)
    return scaler.transform(X)

def t_sne_plot(X, y, title='t-SNE plot'):
    from sklearn.manifold import TSNE
    X_tsne = TSNE(n_components=2, random_state=0).fit_transform(X)
    plt.figure(figsize=(10,10))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis', alpha=0.5)
    plt.title(title)
    plt.colorbar(label='Viability')
    plt.show()

In [ ]:
###########################
# Load & Preprocess Features
###########################

# Read the features file (same as before)
df = pd.read_excel('data/media-4.xlsx', sheet_name='all features')

# Drop columns that are not needed and set the index
df = df.drop(columns=['sample_ID','uniprot_id'])
df.set_index('human_symbol', inplace=True)

# Group by index (to average out duplicates, if any)
df = df.groupby(df.index).mean()

# Visualize missing values
msno.bar(df, figsize=(12, 48), sort="ascending", fontsize=12, color='tomato')

# Drop columns with more than 50% missing values
threshold = 0.5
missing_percentage = df.isnull().mean()
columns_to_drop = missing_percentage[missing_percentage > threshold].index
df_dropped = df.drop(columns=columns_to_drop)

# Impute remaining missing values using KNNImputer
original_index = df_dropped.index
imputer = KNNImputer(n_neighbors=5)
df_imputed_array = imputer.fit_transform(df_dropped)
df_imputed = pd.DataFrame(df_imputed_array, columns=df_dropped.columns, index=original_index)

In [ ]:
###############################
# Load & Preprocess the Target
###############################

# Read the new target file for regression (viability)
target_df = pd.read_excel('data/secretome_samples_for_RNAseq_170111.xlsx', sheet_name='all')

# Select only the relevant columns
target_df = target_df[['Gene name', 'Viability (harvest) [%]']]

# Rename columns so that the gene names match the features dataframe index
target_df = target_df.rename(columns={'Gene name': 'human_symbol', 
                                      'Viability (harvest) [%]': 'Viability'})

# Set the index to 'human_symbol'
target_df.set_index('human_symbol', inplace=True)

In [ ]:
######################################
# Merge Features with the Regression Target
######################################

# Join the imputed features with the target viability values
df_merged = df_imputed.join(target_df, how='inner')
print("Merged data shape:", df_merged.shape)

In [ ]:
#############################################
# Explore Balance, Variability, and Correlations
#############################################

# Identify any columns with zero variability (if any)
zero_var_cols = df_merged.columns[df_merged.nunique() == 1]
print("Zero-variability columns:", zero_var_cols)

# Identify and drop highly correlated features (threshold = 0.80)
correlation_threshold = 0.80
correlation_matrix = df_merged.corr(method='pearson')
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(abs(upper[column]) > correlation_threshold)]
print(f'{len(to_drop)} columns to drop due to high correlation:', to_drop)

# Create a list of predictor columns by excluding the target and dropped columns
predictor_list = list(df_merged.columns)
predictor_list.remove('Viability')
predictor_list = [x for x in predictor_list if x not in to_drop]
print("Number of predictors:", len(predictor_list))

# (Optional) Save the predictor list
with open('preprocessed/predictor_list_regression.json', 'w') as f:
    json.dump(predictor_list, f)

# Visualize correlations (features vs. viability)
show_heatmap(df=df_merged, target='Viability', predictor_list=predictor_list, 
             title='Correlation Heatmap (Viability)', figsize=(65,65))

# (Optional) Standardize values and explore with t-SNE
X_stand  = standardize_values(df_merged[predictor_list].values)
t_sne_plot(X_stand, df_merged['Viability'].values, title='t-SNE plot of standardized features')

In [ ]:
######################################
# Data Splitting Strategy (Regression)
######################################

# Note: When dealing with a continuous target, stratification is not applicable.
# We split the merged data into training, validation, calibration, and testing sets.

# Step 1: Split into training (70%) and temporary (30%) sets
train_df, tmp_df = train_test_split(df_merged, test_size=0.3, random_state=1)

# Step 2: Split temporary set into validation (20% of original) and testing (10% of original)
tmp2_df, test_df = train_test_split(tmp_df, test_size=0.33, random_state=1)

# Step 3: Split tmp2_df into validation (10% of original) and calibration (10% of original)
val_df, cal_df = train_test_split(tmp2_df, test_size=0.5, random_state=1)

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Calibration shape:", cal_df.shape)
print("Test shape:", test_df.shape)

In [ ]:
######################################
# Optuna Study for Regression
######################################

# Helper function to get training data (features and continuous target)
def get_data():
    y = train_df['Viability'].values.astype(float)
    X = train_df[predictor_list].values
    return X, y

# Define the objective function for regression.
# We use the coefficient of determination (R²) as the evaluation metric.
def objective(trial: optuna.Trial) -> float:
    X, y = get_data()
    # Split training data into a new training and validation set (no stratification)
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25, random_state=1)
    
    # Suggest model type: choose among four regression approaches.
    model_type = trial.suggest_categorical("model", ["XGBoost", "RandomForest", "Ridge"])
    
    if model_type == "XGBoost":
        # XGBoost regressor parameters
        param = {
            "objective": "reg:squarederror",
            "colsample_bytree": trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0),
            "learning_rate": trial.suggest_float("xgb_learning_rate", 0.01, 0.1),
            "max_depth": trial.suggest_int("xgb_max_depth", 1, 12),
            "n_estimators": trial.suggest_int("xgb_n_estimators", 100, 1000),
            "subsample": trial.suggest_float("xgb_subsample", 0.5, 1.0)
        }
        model = xgb.XGBRegressor(**param)
        model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=False)
        preds = model.predict(valid_x)
        
    elif model_type == "RandomForest":
        # RandomForest regressor parameters
        param = {
            "n_estimators": trial.suggest_int("rf_n_estimators", 100, 1000),
            "max_depth": trial.suggest_int("rf_max_depth", 1, 12),
            "min_samples_split": trial.suggest_int("rf_min_samples_split", 2, 10),
            "min_samples_leaf": trial.suggest_int("rf_min_samples_leaf", 1, 10),
            "bootstrap": trial.suggest_categorical("rf_bootstrap", [True, False])
        }
        model = RandomForestRegressor(**param)
        model.fit(train_x, train_y)
        preds = model.predict(valid_x)
        
    elif model_type == "Ridge":
        # Ridge regression hyperparameter: alpha (regularization strength) and whether to fit an intercept.
        alpha = trial.suggest_float("ridge_alpha", 1e-6, 1e2, log=True)
        fit_intercept = trial.suggest_categorical("ridge_fit_intercept", [True, False])
        model = Ridge(alpha=alpha, fit_intercept=fit_intercept, max_iter=1000)
        model.fit(train_x, train_y)
        preds = model.predict(valid_x)
    
    # Evaluate the model using R² (the higher, the better)
    score = r2_score(valid_y, preds)
    return score

# Create an Optuna study with the goal of maximizing R²
import warnings
warnings.filterwarnings("ignore")

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="maximize"
)
study.optimize(objective, n_trials=500, timeout=3600)

print("Number of finished trials:", len(study.trials))
print("Best trial:")
best_trial = study.best_trial
print("  R² Value: {:.4f}".format(best_trial.value))
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

print("Best parameters saved to best_params_regression.json")

#### Final Model Training and SHAP Analysis for Regression

In [ ]:
import shap
import matplotlib.patches as mpatches

In [ ]:
#############################
# (1) Combine the Datasets
#############################
# Combine training, validation, calibration, and test splits into one full dataset.
full_df = pd.concat([train_df, val_df, cal_df], ignore_index=True)
print("Full dataset shape:", full_df.shape)


In [ ]:
#############################
# (2) Prepare Data for Training
#############################
X_full = full_df[predictor_list].values
y_full = full_df["Viability"].values  # Continuous target

In [ ]:
#############################
# (3) Train the Best XGBoost Regression Model
#############################
# Set up parameters for XGBoost using the best hyperparameters.
# (Make sure the objective is "reg:squarederror" for regression.)
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": best_trial.params["xgb_colsample_bytree"],
    "learning_rate": best_trial.params["xgb_learning_rate"],
    "max_depth": best_trial.params["xgb_max_depth"],
    "n_estimators": best_trial.params["xgb_n_estimators"],
    "subsample": best_trial.params["xgb_subsample"]
}

model = xgb.XGBRegressor(**params)

# Train on the entire dataset
print("Training the XGBoost regressor on the full dataset...")
model.fit(X_full, y_full)
print("Training complete.")

In [ ]:
#############################
# (4) Evaluate on Test Set
#############################

X_test = test_df[predictor_list].values
y_test = test_df["Viability"].values
preds = model.predict(X_test)

from sklearn.metrics import r2_score
r2 = r2_score(y_test, preds)
print(f"XGBoost Regression model R² on test set: {r2:.3f}")

In [ ]:
#############################
# (5) SHAP Analysis
#############################

# Initialize JS visualization (optional, for interactive plots in notebooks)
shap.initjs()

# Create a TreeExplainer for the trained model
explainer = shap.TreeExplainer(model)

# Compute SHAP values on the test set (or you can choose a different subset)
shap_values = explainer.shap_values(X_test)

## (a) SHAP Beeswarm Summary Plot
plt.figure()
plt.title("SHAP Summary Plot (Beeswarm)")
shap.summary_plot(shap_values, X_test, feature_names=predictor_list)

## (b) SHAP Bar Plot of Mean |SHAP Value|
plt.figure()
plt.title("SHAP Feature Importance (Bar Plot)")
shap.summary_plot(shap_values, X_test, feature_names=predictor_list, plot_type="bar")


print("SHAP analysis complete. You can view the interactive force plot in 'results/shap_force_plot.html'.")

In [ ]:
# --- Step 1: Compute Permutation Importance ---
# Calculate permutation importance on the test set.
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=1)

# Create a DataFrame for permutation importance.
perm_df = pd.DataFrame({
    'Feature': predictor_list,
    'Importance': result.importances_mean
}).sort_values(by='Importance', ascending=False)

# --- Step 2: Compute Pearson Correlations ---
# Compute Pearson correlation coefficients between each feature and cell viability.
# full_df is the complete dataset (all splits combined) that contains the "Viability" column.
correlations = full_df[predictor_list + ['Viability']].corr()['Viability'].drop('Viability')

# Add the correlation values to the permutation importance DataFrame.
perm_df['Correlation'] = perm_df['Feature'].map(correlations)

# --- Step 3: Create a Color Mapping Based on Correlation Sign ---
# Green for features positively correlated with viability; red for negatively correlated.
perm_df['Color'] = perm_df['Correlation'].apply(lambda x: 'green' if x > 0 else 'red')

# --- Step 4: Plot the Top 10 Features with a Custom Legend ---
# Subset to the top 10 most important features.
top_features = perm_df.head(20)

# --- Step 5: Define Mapping for Descriptive Feature Names ---
# The keys here are substrings expected in the cleaned feature names.
feature_mapping = {
    'CTDT_CTDT.prop2.Tr1331': 'Transition volume: small/large',
    'AA.comp_AA.Comp_W': 'AAC - Tryptophan',
    'Peptides.R_charge_charge': 'Protein charge',
    'AA.comp_AA.Comp_Q': 'AAC - Glutamine',
    'abundance_PrEST_conc_cf': 'Production yield in yeast',
    'CTDT_CTDT.prop7.Tr1331': 'Transition solvent accessibility: buried/intermediate',
    'abundance_HPA_pancreas': 'Protein level in pancreas',
    'CTDT_CTDT.prop6.Tr1221': 'Transition secondary structure: helix/strand',
    'PSIM_NG': 'N-linked glycosylation',
    'proStab_Tm.agg': 'Stability index',
    'AA.comp_AA.Comp_A': 'AAC - Alanine',
    'CTDT_CTDT.prop2.Tr1221': 'Transition volume: small/med',
    'AA.comp_AA.Comp_N': 'AAC - Asparagine',
    'CTDT_CTDT.prop4.Tr1221': 'Transition polarizability: low/med',
    'AA.comp_AA.Comp_Y': 'AAC - Tyrosine',
    'abundance_HPA_liver': 'Protein level in liver',
    'Peptides.R_aaComp.mole_NonPolar_Mole%': 'Percent nonpolar residues',
    'CTDT_CTDT.prop6.Tr2332': 'Transition secondary structure: strand/coil',
    'CTDC_CTDC.hydrophobicity.Group3': 'AA comp - hydrophobic residues',
    'CTDT_CTDT.prop7.Tr2332': 'Transition solvent accessibility: exposed/intermediate'
}

def map_feature_name(feature):
    # Remove the "sequence_" prefix.
    cleaned = feature.replace('sequence_', '')
    # Remove a leading "protr." if present.
    if cleaned.startswith("protr."):
        cleaned = cleaned[len("protr."):]
    # Check if any key from our mapping appears in the cleaned feature.
    for key, descriptive in feature_mapping.items():
        if key in cleaned:
            return descriptive
    # If no mapping is found, return the cleaned name.
    return cleaned

# Apply the mapping function.
top_features['Feature_mapped'] = top_features['Feature'].apply(map_feature_name)

# --- Step 6: Remap Colors to New Colors for Improved Aesthetics ---
positive_color = 'royalblue'  # For positive correlation.
negative_color = 'orange'     # For negative correlation.
top_features['Color'] = top_features['Color'].apply(lambda x: positive_color if x == 'green' else negative_color)

# --- Step 7: Plot with Custom Y-Axis Labels and Improved Aesthetics ---
plt.figure(figsize=(12, 8))
ax = sns.barplot(
    x='Importance', 
    y='Feature_mapped', 
    data=top_features, 
    palette=top_features['Color'].tolist(),
    edgecolor='black'
)

# Set title and labels with improved font sizes and bold styling.
ax.set_title("Top 10 Permutation Importance with Correlation Direction", fontsize=18, weight='bold')
ax.set_xlabel("Permutation Importance", fontsize=16)
ax.set_ylabel("Feature", fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=14)

# Add a vertical grid for the x-axis.
ax.xaxis.grid(True, linestyle='--', linewidth=0.7, alpha=0.7)
ax.yaxis.grid(False)

# Remove the top and right spines.
sns.despine(ax=ax, top=True, right=True)

# Create custom legend patches.
positive_patch = mpatches.Patch(color=positive_color, label='Positive correlation with viability')
negative_patch = mpatches.Patch(color=negative_color, label='Negative correlation with viability')
plt.legend(handles=[positive_patch, negative_patch], loc='lower right', fontsize=14)

plt.savefig("results/permutation_importance_ML_reg.png", dpi=300, bbox_inches="tight")

plt.tight_layout()
plt.show()

In [ ]:
# Set a Seaborn theme for a clean and modern look.
sns.set_theme(style='whitegrid', context='talk')

# --- Step 1: Compute Pearson Correlations ---
# Calculate Pearson correlation coefficients between each feature and cell viability.
correlations = full_df[predictor_list + ['Viability']].corr()['Viability'].drop('Viability')
correlations = correlations.dropna()
correlations = correlations.sort_values()

# --- Step 2: Subset the Top 10 Negative and Top 10 Positive Correlations ---
top10_negative = correlations.head(10)
top10_positive = correlations.tail(10)

# Combine the two subsets into one Series and sort again.
subset_correlations = pd.concat([top10_negative, top10_positive]).sort_values()

# --- Step 3: Map Feature Names to Descriptive Labels ---
# Define a mapping dictionary for your new features.
feature_mapping = {
    "AA.comp_AA.Comp_I": "AAC - Isoleucine",
    "abundance_HPA_respiratory": "Protein level (respiratory)",
    "AA.comp_AA.Comp_V": "AAC - Valine",
    "abundance_HPA_GI_tract": "Protein level (GI tract)",
    "abundance_HPA_brain": "Protein level (brain)",
    "CTDC_CTDC.secondarystruct.Group2": "Secondary structure: Group2",
    "abundance_GTEx_Artery": "Gene expression (artery)",
    "abundance_HPA_endocrine": "Protein level (endocrine)",
    "AA.comp_AA.Comp_N": "AAC - Asparagine",
    "iPTMnet_Sumoylation": "Sumoylation",
    "PSIM_OG": "O-linked glycosylation",
    "AA.comp_AA.Comp_L": "AAC - Leucine",
    "abundance_PrEST_conc_cf": "Production yield in yeast",
    "CTDT_CTDT.prop6.Tr1331": "Transition secondary structure: helix/strand",
    "PSIM_DSB": "Disulfide bond",
    "CTDT_CTDT.prop5.Tr1331": "Transition solvent accessibility: buried/intermediate"
}

def map_feature_name(feature):
    # Remove the "sequence_" prefix if present.
    cleaned = feature.replace('sequence_', '')
    # Also remove a leading "protr." if present.
    if cleaned.startswith("protr."):
        cleaned = cleaned[len("protr."):]
    # Check if any mapping key is a substring of the cleaned feature name.
    for key, descriptive in feature_mapping.items():
        if key in cleaned:
            return descriptive
    # If no mapping is found, return the cleaned feature name.
    return cleaned

# The subset_correlations index contains the original feature names.
mapped_index = [map_feature_name(feat) for feat in subset_correlations.index]
subset_correlations.index = mapped_index

# --- Step 4: Define Colors and Update the Plot ---
# Define new colors: royalblue for positive correlations and orange for negative.
positive_color = 'royalblue'
negative_color = 'orange'
colors = [positive_color if val > 0 else negative_color for val in subset_correlations]

# --- Step 5: Plot the Correlation Analysis with Improved Aesthetics ---
plt.figure(figsize=(12, 8))
ax = subset_correlations.plot(kind='barh', color=colors, edgecolor='black', linewidth=1.2)
ax.set_title("Top 10 Negative and Top 10 Positive Pearson Correlations\nwith Cell Viability", fontsize=18, weight='bold')
ax.set_xlabel("Correlation Coefficient", fontsize=16)
ax.set_ylabel("Feature", fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.xaxis.grid(True, linestyle='--', linewidth=0.7, alpha=0.7)
ax.yaxis.grid(False)
sns.despine(ax=ax, top=True, right=True)

# --- Step 6: Add a Custom Legend ---
positive_patch = mpatches.Patch(color=positive_color, label='Positive correlation with viability')
negative_patch = mpatches.Patch(color=negative_color, label='Negative correlation with viability')
plt.legend(handles=[positive_patch, negative_patch], loc='lower right', fontsize=14)

plt.savefig("results/corr_viability_protfeatures.png", dpi=300, bbox_inches="tight")

plt.tight_layout()
plt.show()